In [3]:
import networkx as nx
import numpy as np
import random

In [ ]:
# 1a (config model)

def config_model(degseq):
    ''' 
    implements configuration model to generate graph with preset degree distribution

    args:
        degseq (list): degree sequence

    returns:
        nx.Graph(): randomized graph
    '''
    G = nx.Graph()
    ds_copy = list(degseq) # creating a copy of degseq to avoid writing over the original

    # list comprehension to create nodes to populate G with
    nodes_with_attrs = [(i, {"degree": deg}) for i, deg in enumerate(degseq)]
    G.add_nodes_from(nodes_with_attrs)

    node_ids = list(G.nodes())

    while sum(ds_copy) > 0: # until every stub has been matched
        stubs = random.sample(node_ids, 2)
        i, j = stubs
        if ds_copy[i] > 0 and ds_copy[j] > 0:
            G.add_edge(i, j)
            ds_copy[i] -= 1
            ds_copy[j] -= 1
        else:
            continue

SyntaxError: invalid syntax (920292874.py, line 24)

next step connect each node to another if it has the 